In [ ]:
# default_exp camera_capture

# Camera capture

> Video capture and image transformations.

In [1]:
#export
from expoco.core import *
import numpy as np
import cv2, time, math, json
from pathlib import Path

In [2]:
#export
def _update_and_show(image, image_display_helper, text):
    image = cv2.flip(image, 1)
    image = cv2.putText(image, text, (20,40), cv2.FONT_HERSHEY_COMPLEX, 1, (20,20,20), 2)
    image = cv2.putText(image, text, (20,40), cv2.FONT_HERSHEY_COMPLEX, 1, (200,200,200), 1)
    image_display_helper.show(image)

In [3]:
#export
def _countdown(video_capture, image_display_helper, classes):
    for i in range(3,0,-1):
        retval, image = video_capture.read()
        _update_and_show(image, image_display_helper, f'Capture: {classes} in {i}')
        time.sleep(0.75)

In [4]:
#export
def capture_session(classes, stop_after, comments, path='data'):
    "Run a video capture session"
    path = Path(f'{path}/capture_sessions/{now()}')
    path.mkdir(parents=True, exist_ok=True)
    video_capture = cv2.VideoCapture(0)
    width, height = [int(video_capture.get(p)) for p in [cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT]]
    retval, image = video_capture.read()
    image_display_helper = ImageDisplayHelper(cv2.flip(image, 1), 'expoco: Capture session')
    try:
        _countdown(video_capture, image_display_helper, classes)
        metadata = dict(count=0, stop_after=stop_after, path=path_to_str(path), classes=classes, 
                        capture_width=width, capture_height=height, start_date=now(), 
                        comments=comments) # TODO: add relevant software versions etc
        for capture_count in range(1, stop_after+1):
            retval, image = video_capture.read()
            if not retval:
                raise Exception('Failed to read from video capture')
            assert cv2.imwrite(f'{path}/{capture_count}.png', image), 'Failed to save image'
            _update_and_show(image, image_display_helper, f'{classes}: {capture_count}')
            time.sleep(.05)
        metadata['count'] = capture_count
        metadata['end_date'] = now()
        with open(path/'metadata.json', 'w') as f: json.dump(metadata, f, indent=2)
    finally:
        video_capture.release()
    return path

In [7]:
#do_not_test
_capture_session_path = capture_session('NO_EXPRESSION', 5, 'do not keep', 'test/data')
_capture_session_path

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x80\x00\x00\x01\xe0\x08\x02\x00\x00\x00\xba\xb3K…

WindowsPath('test/data/capture_sessions/20220201_111846')

In [8]:
#do_not_test
import shutil
shutil.rmtree(_capture_session_path)

In [ ]:
#export
def get_viseme_class(metadata, viseme_classes=['NO_EXPRESSION', 'OO', 'EE', 'AH', 'RANDOM_TALK']):
    classes = metadata['classes'].split(' ') # TODO: this should be called labels
    result = None
    for c in classes:
        if c in viseme_classes:
            if result is not None:
                raise Exception(f'Found multiple viseme classes in {classes}. Expected 1')
            result = c
    if result is None:
        raise Exception(f'Viseme class not found in {classes}')
    return result

In [ ]:
try:
    get_viseme_class({})
    assert False, 'we should get a KeyError when metadata does not have a classes entry'
except KeyError:
    pass

try:
    get_viseme_class({'classes': None})
    assert False, 'we should get an error classes is not a string'
except:
    pass

try:
    get_viseme_class({'classes': 'no viseme classes here'})
    assert False, 'we should get an error when no viseme classes can be found'
except Exception as ex:
    assert str(ex).startswith('Viseme class not found')

assert 'NO_EXPRESSION' == get_viseme_class({'classes': 'NO_EXPRESSION'})
assert 'NO_EXPRESSION' == get_viseme_class({'classes': 'NO_EXPRESSION CAN_HAVE_OTHER_CLASSES'})
assert 'OO' == get_viseme_class({'classes': 'OO'})

try:
    get_viseme_class({'classes': 'NO_EXPRESSION OO'})
    assert False, 'we should get an error when multiple viseme classes are found'
except Exception as ex:
    assert str(ex).startswith('Found multiple viseme classes')

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()